# Example: Multiple Tables
In this example, I show how doctable can be used with multiple relational tables to perform queries which automatically merge different aspects of your dataset when you use `.select()`. By integrating these relations into the schema, your database can automatically maintain consistency between tables by deleting irrelevant elements when their relations disappear. There are two important features of any multi-table schema using doctable:

(1) Set the foreign_keys=True in the original doctable or ConnectEngine constructor. It is enabled by default though. Otherwise sqlalchemy will not enable.

(2) Use the "foreignkey" column type to set the constraint, probably with the onupdate and ondelete keywords specifiied.

The example I will use here will consist of the following tables:

+ *Books*: keeps titles and publication year of each book. Has a many-to-one relationship with `Authors` and a many-to-many relationship to *Libraries*.
+ *Authors*: keeps author information. Has a one-to-many relationship with `Books`.
+ *Libraries*: information about libraries. Has a many-to-many relationship with `Books`.

This will be our main point of access, so the extra tables are to avoid replicated data and allow us to quickly query by author or library.

In [1]:
import datetime
import dataclasses

import sys
sys.path.append('..')
import doctable

## Many-to-Many Relationships

The premise is that we have an imaginary API where we can get newly released books along with the libraries they are associted with (although they man, in some cases, not have library information). We want to keep track of the set of books with unique titles, and have book information exist on its own (i.e. we can insert book information if it does not have library information). We would also like to keep track of the libraries they belong to. We need this schema to be fast for selection, but it can be slow for insertion.

Primary accesses methods:

+ insert a book
+ query books by year of publication
+ insert a single library and associated books
+ query books associated with libraries in certain zips

In this example, we are going to use two tables with a many-to-many relationships and a table to handle relationships between them (required for a many-to-many relationship):
    
+ *`Books`*: keeps title and publication year of each book. Should exist independently of Libraries, because we may not want to use Libraries at all.
+ *`Libraries`*: keeps name of library, makes it easy to query by Library.
+ *`BookLibraryRelations`*: keeps track of relationships between Books and Libraries.

First we define the `Books` table. Because we are primarily interested in Books, we will create a separate `Book` object for working with them.

In [2]:
@doctable.schema(frozen=True, eq=True)
class Book:
    __slots__ = []
    _id: int = doctable.IDCol()
    isbn: str = doctable.Col(unique=True)
    title: str = doctable.Col()
    year: int = doctable.Col()
    date_updated: datetime.datetime = doctable.UpdatedCol()

class Books(doctable.DocTable):
    _tabname_ = 'books'
    _schema_ = Book
    _indices_ = [doctable.Index('isbn_index', 'isbn')]
    
book_table = Books(target=':memory:')

We are not planning to work with author data outside of the schema definition, so we include it as part of the table definition.

In [3]:
@doctable.schema(frozen=True, eq=True)
class Library:
    __slots__ = []
    _id: int = doctable.IDCol()
    name: str = doctable.Col()
    zip: int = doctable.Col()

class Libraries(doctable.DocTable):
    _tabname_ = 'libraries'
    _schema_ = Library    
    _constraints_ = [doctable.Constraint('unique', 'name', 'zip')]
    

library_table = Libraries(engine=book_table.engine)

In [4]:
class BookLibraryRelations(doctable.DocTable):
    '''Link between books and libraries.'''
    _tabname_ = 'book_library_relations'
    
    @doctable.schema
    class _schema_:
        __slots__ = []
        _id: int = doctable.IDCol()
        book_isbn: int = doctable.Col(nullable=False)
        library_id: int = doctable.Col(nullable=False)
    
    _constraints_ = (
        doctable.Constraint('foreignkey', ('book_isbn',), ('books.isbn',)),
        doctable.Constraint('foreignkey', ('library_id',), ('libraries._id',)),
        doctable.Constraint('unique', 'book_isbn', 'library_id'),
    )

relations_table = BookLibraryRelations(engine=book_table.engine)
relations_table.list_tables()

['book_library_relations', 'books', 'libraries']

Now we have a bunch of libraries sending us their books, and we also have random books being added to our database that are not at libraries.

In [5]:
newly_published_books = [
    Book(isbn='A', title='A', year=2020),
    Book(isbn='E', title='E', year=2018),
]
new_library_books = {
    Library(name='Library1', zip=12345): [
        Book(isbn='A', title='A', year=2020),
        Book(isbn='B', title='B', year=2020),
    ],
    Library(name='Library2', zip=12345): [
        Book(isbn='A', title='A', year=2020),
        Book(isbn='C', title='C', year=2021),
    ],
    Library(name='Library3', zip=67890): [
        Book(isbn='A', title='A', year=2020),
        Book(isbn='B', title='B', year=2020),
        Book(isbn='D', title='D', year=2019),
    ],
}
for book in newly_published_books:
    print(book)

Book(isbn='A', title='A', year=2020)
Book(isbn='E', title='E', year=2018)


Now we insert the list of books that were published. It works as expected.

In [6]:
book_table.insert(newly_published_books, ifnotunique='replace')
book_table.head()

,_id,isbn,title,year,date_updated
0,1,A,A,2020,2022-07-26 20:06:30.100029
1,2,E,E,2018,2022-07-26 20:06:30.100034


In [7]:
for library, books in new_library_books.items():
    r = library_table.insert(library, ifnotunique='ignore')
    book_table.insert(books, ifnotunique='replace')
    relations_table.insert([{'book_isbn':b.isbn, 'library_id': r.lastrowid} for b in books], ifnotunique='ignore')

In [8]:
book_table.select_df()

,_id,isbn,title,year,date_updated
0,2,E,E,2018,2022-07-26 20:06:30.100034
1,6,C,C,2021,2022-07-26 20:06:30.156971
2,7,A,A,2020,2022-07-26 20:06:30.158280
3,8,B,B,2020,2022-07-26 20:06:30.158285
4,9,D,D,2019,2022-07-26 20:06:30.158288


In [9]:
library_table.select_df()

,_id,name,zip
0,1,Library1,12345
1,2,Library2,12345
2,3,Library3,67890


In [10]:
relations_table.select_df()

,_id,book_isbn,library_id
0,1,A,1
1,2,B,1
2,3,A,2
3,4,C,2
4,5,A,3
5,6,B,3
6,7,D,3


## Select Queries That Join Tables

Similar to sqlalchemy, `DocTable` joins are doen simply by replacing the where conditional. While not technically nessecary, typically you will be joining tables on foreign key columns because it is much faster.

In [11]:
bt, lt, rt = book_table, library_table, relations_table

For the first example, say we want to get the isbn numbers of books associated with each library in zip code 12345. We implement the join using a simple conditional  equating the associated keys in each table. Our database schema already knows that the foreign keys are in place, so this expression will give us the join we want.

In [12]:
lt.select([lt['name'], rt['book_isbn']], where=(lt['_id']==rt['library_id']) & (lt['zip']==12345), as_dataclass=False)
#rt.select(where=(lt['_id']==rt['library_id']) & (lt['zip']==12345))

[('Library1', 'A'), ('Library1', 'B'), ('Library2', 'A'), ('Library2', 'C')]

Now say we want to characterize each library according to the age distribution of it's books. We use two conditionals for the join: one connecting library table to relations table, and another connecting relations table to books table. We also include the condition to get only libraries associated with the given zip.

In [13]:
conditions = (bt['isbn']==rt['book_isbn']) & (rt['library_id']==lt['_id']) & (lt['zip']==12345)
bt.select([bt['title'], bt['year'], lt['name']], where=conditions, as_dataclass=False)

[('C', 2021, 'Library2'),
 ('A', 2020, 'Library1'),
 ('A', 2020, 'Library2'),
 ('B', 2020, 'Library1')]

In [14]:

inner_tab = lt.table.select().where(bt[''])
#bt.select(where=())
#jt = lt.join(rt, (lt['zip']==12345) & (lt['_id']==rt['library_id']), isouter=False)
print(lt.execute(inner_tab))

KeyError: ''

In [ ]:
jt = lt.join(rt, (lt['zip']==12345) & (lt['_id']==rt['library_id']), isouter=False)
bt.select(where=bt['isbn']==jt.c['book_library_relations_book_isbn'], as_dataclass=True)
bt.select([bt['title'], jt.c['book_library_relations_library_id']], where=bt['isbn']==jt.c['book_library_relations_book_isbn'], as_dataclass=False)
dir(jt.c)
bt.select([bt['title'], jt.c['libraries_name']], where=bt['isbn']==jt.c['book_library_relations_book_isbn'], as_dataclass=False)

In [ ]:
# define a test dataset
collection = (
    ('Devin Cornell', 'green', 'The Case of Austerity'),
    ('Devin Cornell', 'green', 'Gender Stereotypes'),
    ('Devin Cornell', 'green', 'Colombian Politics'),
    ('Pierre Bourdieu', 'orange', 'Distinction'),
    ('Pierre Bourdieu', 'orange', 'Symbolic Power'),
    ('Jean-Luc Picard', 'red', 'Enterprise Stories'),
)

In [ ]:
class Authors(doctable.DocTable):
    _tabname_ = 'authors'
    
    @doctable.schema
    class _schema_:
        __slots__ = []
        _id: int = doctable.IDCol()
        name: str = doctable.Col(unique=True)
        fav_color: str = doctable.Col()
        date_updated: datetime.datetime = doctable.UpdatedCol()
    
    _indices_ = (
        doctable.Index('ind_author_name', 'name'),
    )

In [ ]:
class Library(doctable.DocTable):
    _tabname_ = 'library'
    
    @doctable.schema
    class _schema_:
        __slots__ = []
        _id: int = doctable.IDCol()
        name: str = doctable.Col(unique=True)
        address: str = doctable.Col()
        date_updated: datetime.datetime = doctable.UpdatedCol()
            
class LibraryMembership(doctable.DocTable):
    _tabname_ = 'library'
    
    @doctable.schema
    class _schema_:
        __slots__ = []
        _id: int = doctable.IDCol()
        name: str = doctable.Col(unique=True)
        address: str = doctable.Col()
        date_updated: datetime.datetime = doctable.UpdatedCol()

In [ ]:
_constraints_ = (
        doctable.Constraint('foreignkey', ('authname',), ('authors.name',), onupdate="CASCADE", ondelete="CASCADE"),
    )

In [ ]:
# see that both are registered with the engine metadata
adb.engine.tables.keys()

In [ ]:
# define a test dataset
collection = (
    ('Devin Cornell', 'green', 'The Case of Austerity'),
    ('Devin Cornell', 'green', 'Gender Stereotypes'),
    ('Devin Cornell', 'green', 'Colombian Politics'),
    ('Pierre Bourdieu', 'orange', 'Distinction'),
    ('Pierre Bourdieu', 'orange', 'Symbolic Power'),
    ('Jean-Luc Picard', 'red', 'Enterprise Stories'),
)

In [ ]:
for auth, color, title in collection:
    adb.insert({'name':auth, 'fav_color': color}, ifnotunique='ignore')
    bdb.insert({'authname':auth, 'title': title}, ifnotunique='ignore')
adb.count(), bdb.count()

In [ ]:
adb.head()

In [ ]:
bdb.head(10)

## Joint Select Statements
You can perform joins by using select queries with column objects from different tables.

In [ ]:
# this is a left join
bdb.select(['title', adb['name'], adb['fav_color']], where=bdb['authname']==adb['name'], as_dataclass=False)

In [ ]:
# with tables reversed, still returns same output
adb.select(['name', bdb['title']], where=adb['name']==bdb['authname'], as_dataclass=False)

## Cascade deletion
See now that by deleting the author "Devin Cornell", we also removed the corresponding rows in the book table.

In [ ]:
adb.delete(where=adb['name']=='Devin Cornell')

In [ ]:
adb.head()

In [ ]:
bdb.head(10)